In [ ]:
from src.libs.lib import *
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df_original = pd.read_csv("src/data/tabela_ocorrencias_dbpx_com_a_pontuação_academia.csv")

#2.6s

In [ ]:
import pandas as pd
import io

def gerar_metadados_df(df: pd.DataFrame, amostragem: int = 5):
    print("=== DTYPE DAS COLUNAS ===")
    print(df.dtypes)
    
    print("\n=== DESCRIBE (INCLUINDO OBJETOS) ===")
    try:
        describe_df = df.describe(include='all', datetime_is_numeric=True).transpose()
    except TypeError:
        describe_df = df.describe(include='all').transpose()
    print(describe_df)

    print("\n=== PRIMEIRAS LINHAS ===")
    print(df.head(amostragem))

    print("\n=== INFO COMPLETA ===")
    buffer = io.StringIO()
    df.info(buf=buffer)
    print(buffer.getvalue())

    print("\n=== VALORES NULOS POR COLUNA ===")
    print(df.isnull().sum())

    print("\n=== AMOSTRA ESTRATIFICADA POR 'description' (se existir) ===")
    if 'description' in df.columns:
        amostra_estratificada = df.groupby('description').apply(lambda x: x.sample(min(len(x), 3), random_state=42)).reset_index(drop=True)
        print(amostra_estratificada)
    else:
        print("Coluna 'description' não encontrada para amostragem estratificada.")

    # Salvando arquivos para facilitar exportação
    df.dtypes.to_csv("dtypes.csv")
    describe_df.to_csv("describe.csv")
    df.head(amostragem).to_csv("head.csv")
    df.isnull().sum().to_csv("missing_values.csv")

    if 'description' in df.columns:
        amostra_estratificada.to_csv("amostra_estratificada.csv")

In [ ]:
gerar_metadados_df(df_original)

In [ ]:
df_original.head(50)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import Normalize
from matplotlib import cm

# seu DataFrame
df = df_original.copy()

# contagem de ocorrências por descrição, em ordem decrescente
counts = df['description'].value_counts()

# prepara colormap
norm = Normalize(vmin=counts.min(), vmax=counts.max())
cmap = cm.get_cmap('Spectral')
colors = cmap(norm(counts.values))

# cria figura e eixos
fig, ax = plt.subplots(figsize=(12, 8))

# plota barras coloridas
bars = ax.bar(counts.index, counts.values, color=colors)

# define escala logarítmica no eixo Y
ax.set_yscale('log')

# labels e título
ax.set_xlabel('Descrição')
ax.set_ylabel('Contagem de Ocorrências (escala log)')
ax.set_title('Contagem de Ocorrências por Descrição')

# rotaciona ticks
plt.xticks(rotation=45, ha='right')

# adiciona barra de cor para referência de valores
sm = cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax)
cbar.set_label('Número de Ocorrências')

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib import cm

df = df_original.copy()

# Garante que a pontuação seja numérica
df['Pontuação'] = pd.to_numeric(df['Pontuação'], errors='coerce').fillna(0)

# Soma dos pontos por descrição, ordenado decrescente
sums = df.groupby('description')['Pontuação'].sum().sort_values(ascending=False)

# Prepara o colormap Spectral com normalização dos valores de soma
norm   = Normalize(vmin=sums.min(), vmax=sums.max())
cmap   = cm.get_cmap('Spectral')
colors = cmap(norm(sums.values))

# Cria figura e eixos
fig, ax = plt.subplots(figsize=(12, 8))

# Plota barras coloridas conforme o total de pontos
bars = ax.bar(sums.index, sums.values, color=colors)

# Define escala logarítmica no eixo Y
ax.set_yscale('log')

# Labels e título
ax.set_xlabel('Descrição')
ax.set_ylabel('Total de Pontos (escala log)')
ax.set_title('Total de Pontos por Descrição')

# Rotaciona e alinha os ticks do eixo x
plt.xticks(rotation=45, ha='right')

# Adiciona colorbar para referência de valores
sm = cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax)
cbar.set_label('Total de Pontos')

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib import cm

# Copia do DataFrame original (já contendo as linhas "Motorista enviado para atualização")
df = df_original.copy()

# Filtra somente as ocorrências de atualização
updates = df[df['description'] == 'Motorista enviado para atualização']

# Conta quantas atualizações por driver_id e pega os top 50
counts = updates['driver_id'].value_counts().sort_values(ascending=False).head(50)

# Prepara colormap Spectral para colorir conforme a contagem
norm = Normalize(vmin=counts.min(), vmax=counts.max())
cmap = cm.get_cmap('Spectral')
colors = cmap(norm(counts.values))

# Cria figura e eixos
fig, ax = plt.subplots(figsize=(15, 10))

# Plota barras coloridas
bars = ax.bar(counts.index.astype(str), counts.values, color=colors)

# Labels e título
ax.set_xlabel('driver_id')
ax.set_ylabel('Contagem de Envios para Atualização')
ax.set_title('Top 50 Motoristas por Envios para Atualização')

# Rotaciona os ticks do eixo x para melhor visualização
plt.xticks(rotation=90)

# Adiciona colorbar
sm = cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax)
cbar.set_label('Número de Envios')

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib import cm

# Copia do DataFrame original
df = df_original.copy()

# Garante que a pontuação seja numérica
df['Pontuação'] = pd.to_numeric(df['Pontuação'], errors='coerce').fillna(0)

# Soma dos pontos por motorista e seleciona os top 50
points_by_driver = (
    df.groupby('driver_id')['Pontuação']
      .sum()
      .sort_values(ascending=False)
      .head(50)
)

# Prepara colormap Spectral
norm = Normalize(vmin=points_by_driver.min(), vmax=points_by_driver.max())
cmap = cm.get_cmap('Spectral')
colors = cmap(norm(points_by_driver.values))

# Cria figura e eixos
fig, ax = plt.subplots(figsize=(15, 10))

# Plota barras coloridas
bars = ax.bar(points_by_driver.index.astype(str), points_by_driver.values, color=colors)

# Labels e título
ax.set_xlabel('driver_id')
ax.set_ylabel('Total de Pontos')
ax.set_title('Top 50 Motoristas por Total de Pontos')

# Rotaciona ticks
plt.xticks(rotation=90)

# Adiciona colorbar para referência dos valores
sm = cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax)
cbar.set_label('Total de Pontos')

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib import cm

# Copia do DataFrame original (já contendo as linhas "Motorista enviado para atualização")
df = df_original.copy()

# Filtra somente as ocorrências de atualização
updates = df[df['description'] == 'Motorista enviado para comitê']

# Conta quantas atualizações por driver_id e pega os top 50
counts = updates['driver_id'].value_counts().sort_values(ascending=False).head(50)

# Prepara colormap Spectral para colorir conforme a contagem
norm = Normalize(vmin=counts.min(), vmax=counts.max())
cmap = cm.get_cmap('Spectral')
colors = cmap(norm(counts.values))

# Cria figura e eixos
fig, ax = plt.subplots(figsize=(15, 10))

# Plota barras coloridas
bars = ax.bar(counts.index.astype(str), counts.values, color=colors)

# Labels e título
ax.set_xlabel('driver_id')
ax.set_ylabel('Contagem de Envios para comitê')
ax.set_title('Top 50 Motoristas por Envios para comitê')

# Rotaciona os ticks do eixo x para melhor visualização
plt.xticks(rotation=90)

# Adiciona colorbar
sm = cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax)
cbar.set_label('Número de Envios')

plt.tight_layout()
plt.show()
